## V2 
### Altair Stuff
* https://stackoverflow.com/questions/62109475/how-to-deselect-drop-down-box-of-altair

In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
from calitp.sql import to_snakecase

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
import altair as alt
import threshold_utils
from shared_utils import calitp_color_palette as cp
from shared_utils import geography_utils, rt_utils, styleguide, utils

In [3]:
import intake
catalog = intake.open_catalog("./catalog_threshold.yml")

In [4]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

### Tests
* Keep name? 

In [5]:
# Keep name?
def clean_trips():
    df = catalog.trips.read()

    subset = [
        "feed_key",
        "name",
        "route_id",
        "direction_id",
        "shape_id",
    ]

    df = df[subset]

    df = df.drop_duplicates().reset_index(drop=True)

    return df

In [6]:
def clean_routelines():
    df = catalog.route_lines.read()

    # Drop shape array key? Drop n_trips?
    df = df.drop(columns=["shape_array_key"])
    df = (df.drop_duplicates()).reset_index(drop=True)

    # Calculate length of geometry
    df = df.assign(actual_route_length=(df.geometry.length))

    return df

In [7]:
def clean_longest_shape():
    df = catalog.longest_shape.read()

    df = df.rename(columns={"route_length": "longest_route_length"})

    return df

### Function
* Best way is using `gtfs_dataset_key` or `name`?

In [8]:
def merge_trips_routes_longest_shape():
    trips = clean_trips()
    crosswalk = catalog.crosswalk.read()
    routelines = clean_routelines()
    longest_shape = clean_longest_shape()

    m1 = (
        trips.merge(
            crosswalk, how="inner", on=["feed_key", "route_id", "name", "direction_id"]
        )
        .merge(routelines, how="inner", on=["feed_key", "shape_id"])
        .merge(
            longest_shape.drop(columns=["geometry"]),
            how="inner",
            on=[
                "feed_key",
                "gtfs_dataset_key",
                "direction_id",
                "route_id",
                "route_dir_identifier",
                "name",
            ],
        )
    )

    # Calculate out proportion of route length against longest.
    m1["route_length_percentage"] = (
        (m1["actual_route_length"] / m1["longest_route_length"]) * 100
    ).astype(int)

    # Count number of segments that appear in the longest shape.
    m1 = (
        m1.groupby(
            [
                "route_id",
                "name",
                "gtfs_dataset_key",
                "route_dir_identifier",
                "shape_id",
                "longest_shape_id",
                "route_length_percentage",
            ]
        )
        .agg({"segment_sequence": "count"})
        .rename(columns={"segment_sequence": "total_segments"})
        .reset_index()
    )

    return m1

In [9]:
def calculate_longest_shape(operator: str):
    """
    Get total segments of the longest shape and
    shape_id's length compared to the longest
    shape_id's length for a route.
    """
    df = merge_trips_routes_longest_shape()

    df = df.loc[df.name == operator].reset_index(drop=True)

    agg = (
        df.groupby(["name", "route_id", "shape_id"])
        .agg({"route_length_percentage": "max"})
        .reset_index()
    )

    return agg

In [10]:
def summary_stats_route_length():
    """
    Get mean, median, max, and min
    of route length for every operator.
    All in one dataframe.
    """
    df = merge_trips_routes_longest_shape()

    df = (
        df.groupby(["name", "route_id", "shape_id"])
        .agg({"route_length_percentage": "max"})
        .reset_index()
    )
    
    # Get summary stats
    df = (
        df.groupby("name")
        .agg({"route_length_percentage": ["mean", "median", "min", "max"]})
        .reset_index()
    )
    
    # Drop index
    df.columns = df.columns.droplevel()
 
    df = df.rename(columns={"": "name",})
    
    # Melt to long df
    df = pd.melt(df, id_vars=["name"], value_vars=["mean", "median", "min", "max"])
    df = df.rename(columns={"value": "route_length_percentage",})
    
    # Title case variable col
    df.variable = df.variable.str.title()
    
    # Round value col
    df['rounded_route_length_percentage'] = ((df.route_length_percentage/100)*10).astype(int)*10
    
    return df

In [11]:
route_length = summary_stats_route_length()

### Operator

* just 1 boxplot or dot plot for the operator for scheduled shapes % route length
* i've also pared back the summary stats to accompany each chart (just the crucial stuff)
* only allow altair selection on operator, not time or segment, i'm not going to remember what it is moving from selection to selection, so i'd rather see it at once. 
    * For usable trips remove time/segment. 
    * Thought the workflow would be running per operator as opposed to running all operators and selecting from the dropdown menu?
    * If the dropdown menu should be operator -> have to combine all the dataframes together.
* if you find it too crowded, i suggest paring down what's * included in SEGMENT_CUTOFFS ....likely we're never going to go up to 0.9, so you can probably scale it back
* continue with the alt.vconcat, but i want the single boxplot or dotplot to be aligned the same way as the bar
the table can be printed below (outside of the charts) if that's easier
    * Wants charts to run in the same direction.
    * Rotate dotplot instead of increasing up and down. Increasing should be left to right. 
    

* Box plot looks super bare.

#### Shape ID vs. Longest Shape ID Lengths 
* https://stackoverflow.com/questions/75414637/altair-panel-chart-dynamic-dropdown-filter-doesnt-work

In [12]:
def route_len_dotpot(df):
    # https://altair-viz.github.io/gallery/wilkinson-dot-plot.html
    
    df = threshold_utils.clean_up_columns(df)
    
    dropdown_list = df["Name"].sort_values().unique().tolist()
    
    dropdown = alt.binding_select(options=[None] + dropdown_list, labels = ['All'] + dropdown_list, name = "Operator")    
    selection = alt.selection_single(fields=["Name"], bind=dropdown)
    
    chart = (alt.Chart(route_length).mark_circle(opacity=1, size = 200).transform_window(
    id='rank()',
    groupby=['Variable']).encode(
  
    alt.X('Rounded Route Length Percentage:O', sort='ascending', axis=alt.Axis(ticks=False, grid=False)),
          alt.Y('Name:N'), 
    color=alt.Color("Variable:N", scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS), legend=None),
    tooltip = ['Name', 'Variable', 'Route Length Percentage'])
             .properties(title = "Length of Shape ID versus Longest Shape ID").add_selection(selection).transform_filter(selection))
    
    chart = threshold_utils.chart_size(chart, 500, 300)
    display(df)
    return chart

##### Boxplot

In [13]:
# boxplot = merge_trips_routes_longest_shape()

In [14]:
"""
boxplot = (
    boxplot.groupby(["name", "route_id", "shape_id"])
    .agg({"route_length_percentage": "max"})
    .reset_index()
)
"""

'\nboxplot = (\n    boxplot.groupby(["name", "route_id", "shape_id"])\n    .agg({"route_length_percentage": "max"})\n    .reset_index()\n)\n'

In [15]:
"""
threshold_utils.chart_size(
    (
        alt.Chart(boxplot)
        .mark_boxplot(extent="min-max")
        .encode(
            x="route_length_percentage:Q",
            y="name:N",
            color=alt.Color(
                "name",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
                legend=None,
            ),
        )
    ).properties(title=f"Test"),
    1000,
    1000,
)"""

'\nthreshold_utils.chart_size(\n    (\n        alt.Chart(boxplot)\n        .mark_boxplot(extent="min-max")\n        .encode(\n            x="route_length_percentage:Q",\n            y="name:N",\n            color=alt.Color(\n                "name",\n                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),\n                legend=None,\n            ),\n        )\n    ).properties(title=f"Test"),\n    1000,\n    1000,\n)'

#### Trip Time and Segment

In [16]:
def merge_trip_diagnostics_with_total_segments():
    trip_diagnostics = catalog.trip_stats.read()
    segments = catalog.longest_shape.read()

    total_segments_by_shape = (
        segments.groupby(["gtfs_dataset_key", "name", "route_dir_identifier"])
        .segment_sequence.nunique()
        .reset_index()
        .rename(columns={"segment_sequence": "total_segments"})
    )

    df = pd.merge(
        trip_diagnostics,
        total_segments_by_shape,
        on=["gtfs_dataset_key", "route_dir_identifier"],
        how="inner",
        validate="m:1",
    )
    
    df = df.assign(
        pct_vp_segments=df.num_segments_with_vp.divide(df.total_segments),
        trip_time=((df.trip_end - df.trip_start) / np.timedelta64(1, "s") / 60).astype(
            int
        ),
        total_trips=df.groupby(["gtfs_dataset_key", "name"]).trip_id.transform(
            "nunique"
        ),
    )

    return df

In [17]:
all_ops = merge_trip_diagnostics_with_total_segments()

In [18]:
def summary_valid_trips_by_cutoff(df, time_cutoffs: list, segment_cutoffs: list):
    final = pd.DataFrame()

    for t in time_cutoffs:
        for s in segment_cutoffs:
            valid = (
                df[(df.trip_time >= t) & (df.pct_vp_segments >= s)]
                .groupby(["gtfs_dataset_key", "name", "total_trips"])
                .trip_id.nunique()
                .reset_index()
                .rename(columns={"trip_id": "n_trips"})
            )

            valid = valid.assign(
                trip_cutoff=t, segment_cutoff=s, cutoff=f"{t}+ min & {s*100}%+ segments"
            )

            final = pd.concat([final, valid], axis=0)

    final = final.assign(percentage_usable_trips=final.n_trips.divide(final.total_trips))

    return final

In [19]:
TIME_CUTOFFS = [5, 10, 15]
SEGMENT_CUTOFFS = [
    0.1,
    0.2,
    0.3,
    0.4,
    0.5,
    0.6,
    0.7,
    0.8
]

In [20]:
valid_stats = summary_valid_trips_by_cutoff(all_ops, TIME_CUTOFFS, SEGMENT_CUTOFFS)

In [21]:
def operator_trip_chart(df):
    df = threshold_utils.clean_up_columns(df)

    chart = (
        alt.Chart(df)
        .mark_bar()
        .encode(
            x=alt.X(
                "Percentage Usable Trips:Q",
                sort=alt.SortField("Percentage Usable Trips", order="descending"),
            ),
            y=alt.Y(
                "Cutoff:N", sort=alt.SortField("Percentage Usable Trips", order="descending")
            ),
            color=alt.Color(
                "Cutoff:N", scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS)
            ),
            tooltip=[
                "Name",
                "Cutoff",
                "Percentage Usable Trips",
                "N Trips",
            ],
        )
        .properties(title=f"Percentage of Usable Trips")
        .add_selection(selection1)
        .transform_filter(selection1)
    )

    chart = threshold_utils.chart_size(chart, 500, 300)
    return chart

#### Bind these graphs together.

In [22]:
route_length = threshold_utils.clean_up_columns(route_length)

In [23]:
valid_stats = threshold_utils.clean_up_columns(valid_stats)

In [24]:
# Column that controls the bar charts
dropdown_list = route_length["Name"].sort_values().unique().tolist()

In [25]:
# Dropdown menu 1
dropdown = alt.binding_select(options=[None] + dropdown_list, labels = ['All'] + dropdown_list, name = "Operator")    
selection = alt.selection_single(fields=["Name"], bind=dropdown)
  

#### Create text table

In [26]:
route_length["Zero"] = 0
route_length["Full Information"] = route_length["Variable"] + '-' + route_length["Route Length Percentage"].astype(str)

In [27]:
route_len_table = (
        (alt.Chart(route_length)
            .mark_circle()
            .encode(x=alt.X("Zero:Q", axis=None), 
            y=alt.Y("Full Information", axis=None))
            .properties(title=f"Summary Statistics")).add_selection(selection).transform_filter(selection))
    
route_len_table = (route_len_table.mark_text(
        align="center",
        baseline="middle",
        dx=5)
        .encode(text="Full Information:N")
           )
    

#### Dotplot

In [28]:
route_len_dot = (alt.Chart(route_length).mark_circle(opacity=1, size = 150).transform_window(
    id='rank()',
    groupby=['Variable']).encode(
  
    alt.X('Rounded Route Length Percentage:O', sort='ascending', axis=alt.Axis(ticks=False, grid=False)),
          alt.Y('Name:N'), 
    color=alt.Color("Variable:N", scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS), legend=None),
    tooltip = ['Name', 'Variable', 'Route Length Percentage'])
             .properties(title = "Length of Shape ID versus Longest Shape ID").add_selection(selection).transform_filter(selection))
route_len_dot = threshold_utils.chart_size(route_len_dot, 500, 300)

In [29]:
route_len_table = threshold_utils.chart_size(route_len_table, 500,300)

#### Find most lenient and stringent of each operator

In [30]:
valid_stats_leniency = valid_stats.groupby(["Name"]).agg({"Percentage Usable Trips": ["max", "min"]}).reset_index()

In [31]:
valid_stats_leniency.columns = valid_stats_leniency.columns.droplevel()

In [32]:
valid_stats_leniency = valid_stats_leniency.rename(columns={"": "name",})

In [33]:
valid_stats_leniency = threshold_utils.clean_up_columns(valid_stats_leniency)

In [34]:
valid_stats_leniency["Zero"] = 0

In [35]:
valid_stats_leniency['Full Information'] = 'Most Lenient: ' + valid_stats_leniency.Max.astype(str) + ' Most Stringent: ' + valid_stats_leniency.Min.astype(str)

In [36]:
valid_stats_leniency.sample()

,Name,Max,Min,Zero,Full Information
40,LA Metro Bus Schedule,0.99,0.70,0,Most Lenient: 0.9917463075586447 Most Stringent: 0.6980886185925282


In [60]:
trip_stats_table = (
        (alt.Chart(valid_stats_leniency)
            .mark_circle()
            .encode(x=alt.X("Zero:Q", axis=None), 
            y=alt.Y("Full Information", axis=None))
            .properties(title=f"Summary Statistics")).add_selection(selection).transform_filter(selection))
    
trip_stats_table = (trip_stats_table.mark_text(
        align="center",
        baseline="middle",
        dx=5)
        .encode(text="Full Information:N")
           )
trip_stats_table =  threshold_utils.chart_size(trip_stats_table, 500,300) 

####  Trip Stats

In [38]:
trip_stats_chart = (
        alt.Chart(valid_stats)
        .mark_bar()
        .encode(
            x=alt.X(
                "Percentage Usable Trips:Q",
                sort=alt.SortField("Percentage Usable Trips", order="descending"),
            ),
            y=alt.Y(
                "Cutoff:N", sort=alt.SortField("Percentage Usable Trips", order="descending")
            ),
            color=alt.Color(
                "Cutoff:N", scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS), legend=None
            ),
            tooltip=[
                "Name",
                "Cutoff",
                "Percentage Usable Trips",
                "N Trips",
            ],
        )
        .properties(title=f"Percentage of Usable Trips")
        .add_selection(selection)
        .transform_filter(selection)
    )
trip_stats_chart = threshold_utils.chart_size(trip_stats_chart, 500, 300)

In [61]:
route_len_dot & route_len_table & trip_stats_chart & trip_stats_table

alt.VConcatChart(...)

In [40]:
routelengthlist = set(route_length.Name.unique().tolist())
tripslist = set(valid_stats.Name.unique().tolist())
routelengthlist - tripslist

{'Bay Area 511 ACE Schedule',
 'Bay Area 511 BART Schedule',
 'Bay Area 511 Capitol Corridor Schedule',
 'Bay Area 511 Commute.org Schedule',
 'Bay Area 511 Golden Gate Ferry Schedule',
 'Bay Area 511 Rio Vista Delta Breeze Schedule',
 'Bay Area 511 SFO AirTrain Schedule',
 'Bay Area 511 South San Francisco Shuttle Schedule',
 'Bay Area 511 Treasure Island Ferry Schedule',
 'Bay Area 511 Vacaville City Coach Schedule',
 'Merced GMV Schedule',
 'Sacramento Schedule',
 'TIME GMV Schedule',
 'eTrans Schedule'}

In [41]:
tripslist - routelengthlist 

set()

### Statewide
#### Statewide Routes by Cutoff

In [42]:
trips_routes_shape = merge_trips_routes_longest_shape()

In [43]:
trip_stats = catalog.trip_stats.read()

In [44]:
# trip_stats.columns, trips_routes_shape.columns

In [45]:
def statewide_summary():
    trips_routes_shape = merge_trips_routes_longest_shape()
    trip_stats = catalog.trip_stats.read()

    m1 = trip_stats.merge(
        trips_routes_shape.drop(columns=["route_length_percentage"]),
        how="inner",
        on=["gtfs_dataset_key", "route_dir_identifier"],
    )

    m1 = m1.assign(
        pct_vp_segments=m1.num_segments_with_vp.divide(m1.total_segments),
        trip_time=((m1.trip_end - m1.trip_start) / np.timedelta64(1, "s")) / 60,
        total_routes=m1.groupby("route_id").trip_id.transform("nunique"),
    )

    return m1

In [46]:
statewide = statewide_summary()

In [47]:
statewide.head(3)

,gtfs_dataset_key,_gtfs_dataset_name,trip_id,route_dir_identifier,trip_start,trip_end,num_segments_with_vp,route_id,name,shape_id,longest_shape_id,total_segments,pct_vp_segments,trip_time,total_routes
0,02af2d11f5bd44434c581540e9e857d8,LA DOT VehiclePositions,183-00dqgv6kf,3371121095,2023-01-18 17:02:55+00:00,2023-01-18 17:30:57+00:00,7,4446,LA DOT Schedule,20569,20569,7,1.00,28.03,237
1,02af2d11f5bd44434c581540e9e857d8,LA DOT VehiclePositions,183-04vq6dp71,3371121095,2023-01-18 18:40:40+00:00,2023-01-18 19:10:40+00:00,6,4446,LA DOT Schedule,20569,20569,7,0.86,30.00,237
2,02af2d11f5bd44434c581540e9e857d8,LA DOT VehiclePositions,183-0541i4kgy,3371121095,2023-01-18 16:07:58+00:00,2023-01-18 16:34:01+00:00,7,4446,LA DOT Schedule,20569,20569,7,1.00,26.05,237


In [48]:
def route_thresholds_time_only(df, time_cutoffs: list):

    total_unique_routes = df.route_id.nunique()

    # Find stats for all operators
    all_operators = pd.DataFrame()
    for t in time_cutoffs:
        valid = (
            df[(df.trip_time >= t)][["route_id"]]
            .nunique()
            .reset_index()
            .rename(columns={0: "Total Routes in Category"})
        )

        valid = valid.assign(route_cutoff=t, cutoff=f"{t}+ min")

        all_operators = pd.concat([all_operators, valid], axis=0)

    all_operators = all_operators.assign(
        total_routes=total_unique_routes,
        pct_usable_routes=all_operators["Total Routes in Category"].divide(
            total_unique_routes
        ),
        calitp_itp_id="all operators",
    )

    return all_operators

In [49]:
statewide_thresholds_time_only = route_thresholds_time_only(statewide, TIME_CUTOFFS)

In [50]:
statewide_thresholds_time_only.sample()

,index,Total Routes in Category,route_cutoff,cutoff,total_routes,pct_usable_routes,calitp_itp_id
0,route_id,1012,10,10+ min,1022,0.99,all operators


In [51]:
threshold_utils.chart_size((
    alt.Chart(statewide_thresholds_time_only)
    .mark_bar()
    .encode(
        x=alt.X(
            "cutoff:N",
            sort=alt.SortField("pct_usable_routes", order="descending"),
        ),
        y=alt.Y(
            "pct_usable_routes:Q",
            sort=alt.SortField("pct_usable_routes", order="descending"),
        ),
        color=alt.Color(
            "cutoff:N", scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS)
        ),
        tooltip=[
            "total_routes",
            "cutoff",
            "Total Routes in Category",
            "pct_usable_routes",
        ],
    )
    .properties(title=f"Percentage of Usable Routes for the State")
), 500, 300)

alt.Chart(...)

In [52]:
def route_thresholds(df, time_cutoffs: list, segment_cutoffs: list):

    total_unique_routes = df.route_id.nunique()

    # Find stats for all operators
    all_operators = pd.DataFrame()

    for t in time_cutoffs:
        for s in segment_cutoffs:
            valid = (
                df[(df.trip_time >= t) & (df.pct_vp_segments >= s)][["route_id"]]
                .nunique()
                .reset_index()
                .rename(columns={0: "Total Routes in Category"})
            )

            valid = valid.assign(
                time_cutoff=t,
                segment_cutoff=s,
                route_cutoff=f"{t}+ min & {s*100}%+ segments",
                total_routes=total_unique_routes,
                pct_usable_routes=valid["Total Routes in Category"].divide(
                    total_unique_routes
                ),
            )

            all_operators = pd.concat([all_operators, valid], axis=0)

    return all_operators

In [53]:
statewide_thresholds = route_thresholds(statewide, TIME_CUTOFFS, SEGMENT_CUTOFFS)

In [54]:
(
    alt.Chart(statewide_thresholds)
    .mark_bar()
    .encode(
        x=alt.X(
            "pct_usable_routes:Q",
            sort=alt.SortField("pct_usable_routes", order="descending"),
        ),
        y=alt.Y(
            "route_cutoff:N",
            sort=alt.SortField("pct_usable_routes", order="descending"),
        ),
        color=alt.Color(
            "route_cutoff:N", scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS)
        ),
        tooltip=[
            "total_routes",
            "route_cutoff",
            "Total Routes in Category",
            "pct_usable_routes",
        ],
    )
    .properties(title=f"Percentage of Usable Routes for the State")
)

alt.Chart(...)

#### Route Thresholds by Operators

In [55]:
# Find stats for all operators
operator_df = pd.DataFrame()

In [56]:
# Subset statewide to only 2 operators
subset = statewide[statewide["name"].isin(['Big Blue Bus Schedule','Bay Area 511 AC Transit Schedule',])].reset_index(drop = True)

In [57]:
for o in subset.name.unique().tolist():
    for t in TIME_CUTOFFS:
        for s in SEGMENT_CUTOFFS:
            valid = subset.loc[subset.name == o].reset_index(drop = True)
            total_unique_routes = valid.route_id.nunique() 
            operator = valid.head(1).iloc[0]["name"]
            valid = (
                    valid[(valid.trip_time >= t) & (valid.pct_vp_segments >= s)][["route_id"]]
                    .nunique()
                    .reset_index()
                    .rename(columns={0: "Total Routes in Category"})
                )

            valid = valid.assign(
                time_cutoff=t,
                segment_cutoff=s,
                name = operator,
                route_cutoff=f"{t}+ min & {s*100}%+ segments",
                total_routes=total_unique_routes,
                pct_usable_routes=valid["Total Routes in Category"].divide(
                    total_unique_routes
                ),
            )

            operator_df = pd.concat([operator_df, valid], axis=0)

In [58]:
empty_df.name.sort_values().unique().tolist()

NameError: name 'empty_df' is not defined

In [ ]:
# Dropdown menu 1
dropdown1 = alt.binding_select(
        options=empty_df["name"].sort_values().unique().tolist(), name="name"
    )
selection1 = alt.selection_single(fields=["name"], bind=dropdown1)

(
        alt.Chart(empty_df)
        .mark_bar()
        .encode(
            x=alt.X(
                "pct_usable_routes:Q",
                sort=alt.SortField("pct_usable_routes", order="descending"),
            ),
            y=alt.Y(
                "route_cutoff:N", sort=alt.SortField("pct_usable_routes", order="descending")
            ),
            color=alt.Color(
                "route_cutoff:N", scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS)
            ),
            tooltip=[

            ],
        )
        .properties(title=f"Percentage of Usable Routes")
    )#.add_selection(selection1).transform_filter(selection1)

In [ ]:
# Find stats for all operators
operator_df2 = pd.DataFrame()

In [ ]:
for o in statewide.name.unique().tolist():
    for t in TIME_CUTOFFS:
            valid = statewide.loc[statewide.name == o].reset_index(drop = True)
            total_unique_routes = valid.route_id.nunique() 
            operator = valid.iloc[0]["name"]
            valid = (
                    valid[(valid.trip_time >= t)][["route_id"]]
                    .nunique()
                    .reset_index()
                    .rename(columns={0: "Total Routes in Category"})
                )

            valid = valid.assign(
                time_cutoff=t,
                segment_cutoff=s,
                name = operator,
                route_cutoff=f"{t}+ min",
                total_routes=total_unique_routes,
                pct_usable_routes=valid["Total Routes in Category"].divide(
                    total_unique_routes
                ),
            )

            operator_df2 = pd.concat([operator_df2, valid], axis=0)

In [ ]:
def operator_trip_chart(df):
    df = threshold_utils.clean_up_columns(df)

    # Dropdown menu 1
    dropdown1 = alt.binding_select(
        options=df["Name"].sort_values().unique().tolist(), name="Name"
    )
    selection1 = alt.selection_single(fields=["Name"], bind=dropdown1)

    chart = (
        alt.Chart(df)
        .mark_bar()
        .encode(
            x=alt.X(
                "Pct Usable Routes:Q",
                sort=alt.SortField("Pct Usable Routes", order="descending"),
            ),
            y=alt.Y(
                "Route Cutoff:N", sort=alt.SortField("Pct Usable Routes", order="descending")
            ),
            color=alt.Color(
                "Route Cutoff:N", scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS)
            ),
            tooltip=[
                "Name",
                "Route Cutoff",
                "Pct Usable Routes",
                "Total Routes",
                "Total Routes In Category"
            ],
        )
        .properties(title=f"Percentage of Usable Trips")
        .add_selection(selection1)
        .transform_filter(selection1)
    )

    chart = threshold_utils.chart_size(chart, 500, 300)
    return chart

In [ ]:
operator_trip_chart(operator_df2)